In [19]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import numpy as np
import pandas as pd
import cvxpy as cp
import torch
from copy import copy, deepcopy


In [20]:
np.random.seed(42)

In [21]:
n=100

In [22]:
ds = []

In [23]:
a_min = 0.8
a_var = 0.2

b_min = 0.1
b_var = 0.9

r_min = 0.1
r_var = 0.9

x_0_min = 15.0
x_0_var = 15.0

x_ref_min = 20.0
x_ref_var = 5.0

In [24]:
for i in range(10000):
	
	a=a_var*np.random.rand() + a_min
	b=b_var*np.random.rand() + b_min
	r = r_var*np.random.rand() + r_min
	x_0=  x_0_var*np.random.rand() + x_0_min
	x_ref = x_ref_var*np.array([np.random.rand()]*(n)) + x_ref_min
	

	L = r * np.identity(n)
	I = np.identity(n)
	A = np.array([a**(i+1) for i in range(n)])
	B = np.array([[a**(i-1) * b for i in range(j, 0, -1)] + [0]*(n-j) for j in range(1, n+1)])

	u = cp.Variable(n)
	constraints = [u <= np.ones(n)*10.0, u >= np.zeros(n), cp.abs(u[1:] - u[:-1]) <= np.ones(n-1)*1.0]
	prob = cp.Problem(cp.Minimize((cp.quad_form(A*x_0 +B@u - x_ref, I) + cp.quad_form(u, L))/n),constraints)
	prob.solve()
	if prob.status == 'optimal': 
		#X = np.array([2*(a - a_min)/a_var -1, 2*(b - b_min)/b_var - 1, 2*(r - r_min)/r_var - 1, 2*(x_0-x_0_min)/x_0_var-1, 2*(x_ref[0]-x_ref_min)/x_ref_var -1])
		X = np.array([a, b, r, x_0, x_ref[0]])
		ds.append([X, u.value])
	

5.439624609623831
281.52175519849243
14.629877303332126
44.54837118521577
20.34423916485288
5.096357148616149
8.963201325124551
0.9845320815051812
6.085827126144494
7.4115872856216125
3.113576981214708
2.1163881626463747
43.348693441040766
76.25372002201472
1.5594553234628856
0.3289313607833035
0.5736948102111995
10.277418581463321
23.92085142674507
6.726921928622777
18.54389431935049
30.542626532756977
122.09545314342478
0.8436967996361596
0.4204213536024223
8.865991084346934
19.177836199173047
25.123552214752404
0.22607779937465192
21.65652299360393
0.6018223241513703
19.979576783175794
9.573879820384814
1.7439779779910645
83.13895953339475
9.714550230839253
119.86211034412227
1.7453677667076872
16.2584524068776
14.705854740574058
19.664204169385037
189.23927429732913
6.249022201029687
59.966763097414
1.0575314133457694
5.702701309634888
1.4555526535969343
1.3349270073357216
0.056269690552321346
1.4360296861748651
34.15476321407963
2.626309038796179
0.8258313755605677
1.7326256717390

KeyboardInterrupt: 

In [16]:
cp.abs(u[1:] - u[:-1]).value

array([8.88423599e-02, 6.16548642e-02, 4.27872726e-02, 2.96935321e-02,
       2.06067317e-02, 1.43006694e-02, 9.92438523e-03, 6.88732939e-03,
       4.77967189e-03, 3.31699874e-03, 2.30193212e-03, 1.59749571e-03,
       1.10863061e-03, 7.69367788e-04, 5.33926029e-04, 3.70534061e-04,
       2.57143242e-04, 1.78452242e-04, 1.23842240e-04, 8.59439774e-05,
       5.96433358e-05, 4.13912156e-05, 2.87246112e-05, 1.99342447e-05,
       1.38339109e-05, 9.60040658e-06, 6.66244488e-06, 4.62356420e-06,
       3.20862793e-06, 2.22669700e-06, 1.54526370e-06, 1.07237097e-06,
       7.44202739e-07, 5.16473147e-07, 3.58449747e-07, 2.48806891e-07,
       1.72747525e-07, 1.20006635e-07, 8.34662961e-08, 5.81943649e-08,
       4.07796907e-08, 2.88715025e-08, 2.08619015e-08, 1.56694249e-08,
       1.25926221e-08, 1.12155938e-08, 1.13518039e-08, 1.30189655e-08,
       1.64413354e-08, 2.20800338e-08, 3.06951167e-08, 4.34483285e-08,
       6.20597724e-08, 8.90398706e-08, 1.28028377e-07, 1.84285296e-07,
      

In [17]:
[sol[1] for sol in ds]

[array([0.84985206, 1.84996017, 2.50089588, 2.62509727, 2.66558428,
        2.67878292, 2.68308561, 2.68448826, 2.68494552, 2.68509459,
        2.68514318, 2.68515902, 2.68516419, 2.68516587, 2.68516642,
        2.6851666 , 2.68516666, 2.68516668, 2.68516668, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68516669, 2.68516669,
        2.68516669, 2.68516669, 2.68516669, 2.68

In [18]:
from pickle import dump
dump(ds, open("mpc.pkl", "wb"))